In [1]:
from sklearn.mixture import GaussianMixture
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import seaborn as sns
import pandas as pd
import folium

## 1. Read data

In [2]:
# should take about 30 sec to read
hyads_df = pd.read_csv("./model_dev_data/hyads_coal.csv", index_col=0)
hyads = hyads_df.values  # as numpy
hyads_df.iloc[:10,:10]

,10001,10002,10003,10004,10005,10006,10007,10008,10009,1001
fid,,,,,,,,,,
10,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.5
1001,39.0,39.0,39.0,39.0,39.0,39.0,39.0,39.0,39.0,29.5
1004,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,25.5
10043,107.0,107.0,107.0,107.0,107.0,107.0,107.0,107.0,107.0,24.5
10071,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,14.5
10075,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,27.0,34.5
1008,33.5,33.5,33.5,33.5,33.5,33.5,33.5,33.5,33.5,34.5
1010,42.0,42.0,42.0,42.0,42.0,42.0,42.0,42.0,42.0,34.0
10111,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,45.0,49.5


In [ ]:
hyads1 = hyads.iloc[:,:15000]
hyads2 = hyads.iloc[15001,:30000]
hyads3 = hyads.iloc[30001,:15000]

In [3]:
# rows are power plants
# index is fid (power plant id)
# columns zip codes
hyads_df.shape

(444, 41151)

In [122]:
power_plants = pd.read_csv("./model_dev_data/power_plant_info.csv", index_col=0)
power_plants = power_plants[power_plants.index.isin(hyads_df.index)]
power_plants.shape

(444, 2)

In [123]:
power_plants.head()

,lat,lon
fid,,
3,31.0069,-88.0103
7,34.0128,-85.9708
8,33.6446,-87.2003
10,32.6017,-87.7811
26,33.2442,-86.4567


In [124]:
my_USA_map = './model_dev_data/us-states.json'

In [125]:
fig = folium.Figure(width=630, height=400)
m = folium.Map(location=[39, -96],
               zoom_start=4, 
               zoom_control=False)
for i in range(power_plants.shape[0]):
    loc = power_plants.iloc[i]
    folium.CircleMarker(
        location=loc,
        radius=1.5,
        color="red",
        opacity=0.5
    ).add_to(m)
fig.add_child(m)
m

## 2. Mixture Model

For now, let's fix the number of components

In [260]:
n_comps = 10
n_plants = hyads.shape[0]
n_zip_samples = 10000
samples = np.random.choice(hyads.shape[1], size=n_zip_samples)
hyads0 = hyads[:, samples]

In [ ]:
gmm = GaussianMixture(
    n_components=n_comps,
    covariance_type="full")
gmm.fit(hyads0)

In [279]:
phat = gmm.predict_proba(hyads0)
gmm.converged_

True

In [264]:
colorpal = sns.color_palette("husl", n_comps).as_hex()

In [276]:

labels = np.zeros(n_plants, int)
for i in range(n_plants):
    labels[i] = np.random.choice(n_comps, p=phat[i])
print("Num labels: ", len(np.unique(labels)))
    
fig = folium.Figure(width=630, height=400)
m = folium.Map(location=[39, -96],
               zoom_start=4, 
               zoom_control=False)

comps_to_plot = range(n_comps)
comps_to_plot = [9]
for c in comps_to_plot:
    for i in range(n_plants):
        loc = power_plants.iloc[i]
        lab = labels[i]
        if lab == c:
            folium.CircleMarker(
                location=loc,
                radius=5.0,
                color=colorpal[labels[i]],
                stroke=False,
                fill=True,
                fill_opacity=0.9
            ).add_to(m)
m.add_to(fig)

Num labels:  10
